### построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

In [24]:
import pandas as pd
import numpy as np

In [25]:
data = pd.read_csv( 'adult.csv' )
data.sample(5)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
45057,39,Private,269323,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
9721,63,Private,190296,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1902,40,United-States,>50K
26190,35,Private,411395,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,36,United-States,<=50K
21405,18,Private,120283,12th,8,Never-married,Sales,Own-child,White,Female,0,0,24,United-States,<=50K
26295,24,Private,185821,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,40,United-States,<=50K


Переведем в бинарный тип признак.

In [26]:
data['income']=data['income'].replace(['>50K','<=50K'],[1,0]).astype('int64')
data['gender']=data['gender'].replace(['Female','Male'],[1,0]).astype('int64')

In [27]:
data.sample(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
24015,21,Private,70261,Some-college,10,Never-married,Other-service,Own-child,White,0,0,0,30,United-States,0
27946,29,Private,196116,Prof-school,15,Divorced,Prof-specialty,Own-child,White,1,2174,0,72,United-States,0
11239,39,Private,74194,Assoc-voc,11,Divorced,Craft-repair,Not-in-family,White,0,0,1721,45,United-States,0
2143,40,Private,87771,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,1,0,0,40,United-States,0
23887,43,Self-emp-not-inc,154785,Bachelors,13,Married-civ-spouse,Machine-op-inspct,Wife,Asian-Pac-Islander,1,0,0,80,Thailand,0
40766,63,?,310396,9th,5,Married-civ-spouse,?,Husband,White,0,5178,0,40,United-States,1
42450,20,Private,247115,Some-college,10,Never-married,Adm-clerical,Unmarried,Black,1,0,0,35,United-States,0
31585,23,Private,126346,9th,5,Never-married,Other-service,Unmarried,Black,1,0,0,30,United-States,0
32711,57,State-gov,254949,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,0,0,0,40,United-States,1
23478,30,Private,211028,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,1,0,0,45,United-States,0


Уберем ненужный столбец образования, так как он уже есть в числовом представлении.

In [28]:
data.drop('education', axis=1, inplace=True)

In [29]:
data.nunique()

age                   74
workclass              9
fnlwgt             28523
educational-num       16
marital-status         7
occupation            15
relationship           6
race                   5
gender                 2
capital-gain         123
capital-loss          99
hours-per-week        96
native-country        42
income                 2
dtype: int64

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   educational-num  48842 non-null  int64 
 4   marital-status   48842 non-null  object
 5   occupation       48842 non-null  object
 6   relationship     48842 non-null  object
 7   race             48842 non-null  object
 8   gender           48842 non-null  int64 
 9   capital-gain     48842 non-null  int64 
 10  capital-loss     48842 non-null  int64 
 11  hours-per-week   48842 non-null  int64 
 12  native-country   48842 non-null  object
 13  income           48842 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 5.2+ MB


#### Попробуем угадать доход

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   educational-num  48842 non-null  int64 
 4   marital-status   48842 non-null  object
 5   occupation       48842 non-null  object
 6   relationship     48842 non-null  object
 7   race             48842 non-null  object
 8   gender           48842 non-null  int64 
 9   capital-gain     48842 non-null  int64 
 10  capital-loss     48842 non-null  int64 
 11  hours-per-week   48842 non-null  int64 
 12  native-country   48842 non-null  object
 13  income           48842 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 5.2+ MB


In [39]:
#Закодируем катег столбцы
X = pd.get_dummies(data)
X.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_?,workclass_Federal-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,0,50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,0,7688,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,1,0,0,30,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
y = X['income']
X.drop('income',inplace=True,axis=1)

#### Строим модельки, получаем скоры, сравниваем

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### Логрег

In [42]:
%%time
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
print(f'log score: {clf.score(X_train, y_train)}')
y_predict_log = clf.predict(X_test)

log score: 0.7945238968341278
Wall time: 394 ms


#### SVM

In [37]:
%%time
clf2 = SVC().fit(X_train, y_train)
print(f'SVM score: {clf2.score(X_train, y_train)}')
y_predict_svc = clf2.predict(X_test)

SVM score: 0.7944627796112944


In [40]:
print(roc_auc_score(y_test, y_predict_log))
print(roc_auc_score(y_test, y_predict_svc))

0.6185618960226571
0.5816137692013177


#### Вывод: из коробки логистическая регрессия чуть лучше показала себя, а также работает быстрее.